In [2]:
import pdal
import sys
import os
from collections import defaultdict
import json
sys.path.append("../PythonScripts")
from glob import glob
import open3d as o3d
import os
import numpy as np
import pandas as pd
import multiprocessing as mp
from pipeline_functions import downscale_las, get_scales
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.spatial import KDTree
sns.set()

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
processors = os.cpu_count() - 1

In [32]:
# ROOT = """/home/sspiegel/CapstoneData/WashingtonDC/OpenDataDC_LAS_Point_Cloud_2020_Block1/Block1/2117.las"""
# ROOT = """/home/sspiegel/CapstoneData/Paris/Toronto_3D/L001.ply"""
ROOT = """/home/sspiegel/CapstoneData/PA/225019325.las"""

## Load in base point cloud

In [33]:
pipeline_json = [
    {
        "type": "readers.las",
        "filename": ROOT
    }
        
]


pipeline = pdal.Pipeline(json.dumps(pipeline_json))

# Execute the pipeline
# This will process the data according to the stages defined in the pipeline
pipeline.execute()

xy = pipeline.arrays[0]

mta = pipeline.metadata



In [46]:
# mta

## Get scaling factors

### Downscale point clouds: Methodology

* Downscale point clouds based on multiscaling factor
* The scaling factor chosen from [Thomas Hugues et. al](https://ieeexplore.ieee.org/document/8490990/)
* Parameters
  * initial radius ($r_0$): 0.1
  * Number of scales ($S$): 8
  * Base exponent for expanding radius ($\gamma$): 2
  * scaling factor of grid ($\rho$): 5
  * radius of sphere for scale $s$: $r_s = r_0 * \gamma^{s}$
  * downsampled voxel size: $\frac{r_s}{\rho}$

* Use multiprocessing to speed up process

In [41]:
scls = get_scales(r_0=0.5, rho=10) # Use base paprameters from function

In [42]:
scls

[(0.5, 0.05),
 (1.0, 0.1),
 (2.0, 0.2),
 (4.0, 0.4),
 (8.0, 0.8),
 (16.0, 1.6),
 (32.0, 3.2),
 (64.0, 6.4)]

In [43]:
scls_grid = [(f"""{ROOT}""",g[1]) for g in scls] # Build arguments for starmap function in multiprocessing

In [45]:
# %%timeit
with mp.Pool(processes=processors) as pool:
    results = pool.starmap(downscale_las, scls_grid)

## Grab downscaled files

In [47]:
glbs = glob("""/home/sspiegel/CapstoneData/PA/*_downsampled_*.las""")

In [48]:
glbs

['/home/sspiegel/CapstoneData/PA/225019325_downsampled_0_05.las',
 '/home/sspiegel/CapstoneData/PA/225019325_downsampled_1_6.las',
 '/home/sspiegel/CapstoneData/PA/225019325_downsampled_3_2.las',
 '/home/sspiegel/CapstoneData/PA/225019325_downsampled_0_1.las',
 '/home/sspiegel/CapstoneData/PA/225019325_downsampled_0_8.las',
 '/home/sspiegel/CapstoneData/PA/225019325_downsampled_0_4.las',
 '/home/sspiegel/CapstoneData/PA/225019325_downsampled_6_4.las',
 '/home/sspiegel/CapstoneData/PA/225019325_downsampled_0_2.las']

## Load in Downsampled point clouds to calculate features

In [49]:
pc_dict = []
for s in scls:
    pc_dict.append({
      "r" : s[0],
     "grid_size" : s[1],
     "file_name" : f"""/home/sspiegel/CapstoneData/PA/225019325_downsampled_{str(s[1]).split('.')[0]}_{str(s[1]).split('.')[1]}.las"""
    })
    


In [50]:
pc_dict

[{'r': 0.5,
  'grid_size': 0.05,
  'file_name': '/home/sspiegel/CapstoneData/PA/225019325_downsampled_0_05.las'},
 {'r': 1.0,
  'grid_size': 0.1,
  'file_name': '/home/sspiegel/CapstoneData/PA/225019325_downsampled_0_1.las'},
 {'r': 2.0,
  'grid_size': 0.2,
  'file_name': '/home/sspiegel/CapstoneData/PA/225019325_downsampled_0_2.las'},
 {'r': 4.0,
  'grid_size': 0.4,
  'file_name': '/home/sspiegel/CapstoneData/PA/225019325_downsampled_0_4.las'},
 {'r': 8.0,
  'grid_size': 0.8,
  'file_name': '/home/sspiegel/CapstoneData/PA/225019325_downsampled_0_8.las'},
 {'r': 16.0,
  'grid_size': 1.6,
  'file_name': '/home/sspiegel/CapstoneData/PA/225019325_downsampled_1_6.las'},
 {'r': 32.0,
  'grid_size': 3.2,
  'file_name': '/home/sspiegel/CapstoneData/PA/225019325_downsampled_3_2.las'},
 {'r': 64.0,
  'grid_size': 6.4,
  'file_name': '/home/sspiegel/CapstoneData/PA/225019325_downsampled_6_4.las'}]

In [76]:
test = pc_dict[0]

pipeline_json = {
    "pipeline" :
[
    {
        "type": "readers.las",
        "filename": test['file_name']
    }
]
}

pipeline = pdal.Pipeline(json.dumps(pipeline_json))

# Execute the pipeline
# This will process the data according to the stages defined in the pipeline
pipeline.execute()

xy_downscaleT = pipeline.arrays[0]
xy_downscaleT = np.vstack((xy_downscaleT['X'], xy_downscaleT['Y'], xy_downscaleT['Z'])).T

In [72]:
xyzT = np.vstack((xy["X"], xy["Y"], xy["Z"])).T

minP = np.min(xyzT, axis = 0)

xyz = xyzT - minP


cls = xy["Classification"]

In [77]:
xy_downscale = xy_downscaleT - minP

In [79]:
test["r"]

0.5

In [80]:
tree_xy = KDTree(xyz)
tree_downscale = KDTree(xy_downscale)

In [81]:
indexes = tree_xy.query_ball_tree(tree_downscale, r=test["r"])


In [88]:
np.linalg.eig(np.cov(xy_downscale[indexes[0]].T))

EigResult(eigenvalues=array([0.0868928 , 0.00063278, 0.00143981]), eigenvectors=array([[ 0.96833624,  0.2496086 ,  0.00452483],
       [ 0.23386776, -0.90063091, -0.366292  ],
       [ 0.08735443, -0.35575203,  0.93048895]]))

In [59]:
# indexes

In [107]:
0.00000001 **(1/3)

0.0021544346900318843

In [108]:
# for ids in indexes:
e_z = np.array([[0.,0.,1]])
featList = []

for i in range(xyz.shape[0]):
    if (i + 1) % 100000 == 0:
        print(f"""Processing point {i + 1}...""")
    test_point = xyz[i].reshape(1, -1)
    surroundedPoint = xy_downscale[indexes[i]]
    N = len(indexes[i])

    if surroundedPoint.shape[0] < 5:
        feat = np.array((0., 0., 0., 0., 0., 0., 0., 0., 0., N))
        featList.append(feat)
    else:
    # if surroundedPoint.ndim < 2:
    #     surroundedPoint = surroundedPoint.reshape(1, -1)
        # print(surroundedPoint.shape)
        if sorted_eigs.max() == 0.:
            print(f"""Error in point {i}: {N}""")
            
        avg =  surroundedPoint - np.mean(surroundedPoint, axis=0)
        cov  = np.cov(avg.T)
        eigs, vecs = np.linalg.eig(cov)
        sort_indices = np.argsort(eigs)
        sorted_eigs = eigs[sort_indices]
        sorted_vecs = vecs[:, sort_indices]
        
        
        
        sum_of_eigs = sorted_eigs.sum()
        omni = sorted_eigs.prod()**(1/3)
        entro = -1 * (sorted_eigs*np.log((sorted_eigs + 1e-9))).sum()
        lin = (sorted_eigs[0] - sorted_eigs[1]) / (sorted_eigs[0] + 1e-9)
        pln = (sorted_eigs[1] - sorted_eigs[2]) / (sorted_eigs[0] + 1e-9)
        sph = sorted_eigs[2] / (sorted_eigs[0] + 1e-9)
        curv = sorted_eigs[2] / np.sum(sorted_eigs)
        vert_1 = np.squeeze(np.abs((np.pi / 2) - np.arccos(sorted_vecs[0].reshape(1,-1)@e_z.T)))
        vert_2 = np.squeeze(np.abs((np.pi / 2) - np.arccos(sorted_vecs[2].reshape(1,-1)@e_z.T)))
        
        
        feat = np.array((sum_of_eigs, omni, entro, lin, pln, sph, curv, vert_1, vert_2, N))
        featList.append(feat)
        

Processing point 100000...
Processing point 200000...
Processing point 300000...
Processing point 400000...
Processing point 500000...
Processing point 600000...
Processing point 700000...
Processing point 800000...
Processing point 900000...
Processing point 1000000...
Processing point 1100000...
Processing point 1200000...
Processing point 1300000...
Processing point 1400000...
Processing point 1500000...
Processing point 1600000...
Processing point 1700000...
Processing point 1800000...
Processing point 1900000...
Processing point 2000000...
Processing point 2100000...
Processing point 2200000...
Processing point 2300000...
Processing point 2400000...
Processing point 2500000...
Processing point 2600000...
Processing point 2700000...
Processing point 2800000...
Processing point 2900000...


/tmp/ipykernel_10967/69674744.py:32: RuntimeWarning: invalid value encountered in scalar power
  omni = sorted_eigs.prod()**(1/3)


Processing point 3000000...
Processing point 3100000...
Processing point 3200000...
Processing point 3300000...
Processing point 3400000...
Processing point 3500000...
Processing point 3600000...
Processing point 3700000...
Processing point 3800000...
Processing point 3900000...
Processing point 4000000...
Processing point 4100000...
Processing point 4200000...
Processing point 4300000...
Processing point 4400000...
Processing point 4500000...
Processing point 4600000...
Processing point 4700000...
Processing point 4800000...
Processing point 4900000...
Processing point 5000000...
Processing point 5100000...
Processing point 5200000...
Processing point 5300000...
Processing point 5400000...
Processing point 5500000...
Processing point 5600000...
Processing point 5700000...
Processing point 5800000...
Processing point 5900000...
Processing point 6000000...
Processing point 6100000...
Processing point 6200000...
Processing point 6300000...
Processing point 6400000...
Processing point 650

In [109]:
featList = np.array(featList)

In [113]:
featList[np.isnan(featList)] = 0.0

In [119]:
featList[:, -1].min()

np.float64(1.0)

array([ 8.45776316e-02,  2.66904429e-03,  2.22837074e-01, -4.03901323e+01,
       -1.04201868e+03,  1.08340880e+03,  9.62346645e-01,  1.31827182e+00,
        3.89195429e-02,  2.00000000e+01])